<a href="https://colab.research.google.com/github/bishram-acharya/hitachi_solution/blob/main/hitachi_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>